In [1]:
import os
import json
import pandas as pd
import numpy as np
from preproc import preproc
import pprint

In [2]:
proj_list = [
    'activiti_dot',
    'fastjson_dot',
    'commons-exec_dot',
    'httpcore_dot',
    'incubator-dubbo_dubbo-remoting.dubbo-remoting-netty',
    'incubator-dubbo_dubbo-rpc.dubbo-rpc-dubbo',
    'rxjava2-extras_dot',
    # 'hutool_hutool-cron',
    'elastic-job-lite_dot',
    'elastic-job-lite_elastic-job-lite-core',
    # 'esper_examples.rfidassetzone',
    # 'yawp_yawp-testing.yawp-testing-appengine',
    'luwak_luwak',
    'fluent-logger-java_dot',
    'delight-nashorn-sandbox_dot',
    'handlebars.java_dot',
    'assertj-core_dot',
    'db-scheduler_dot',
    'http-request_dot',
    'timely_server',
    'ninja_dot',
    'orbit_dot',
    'logback_dot',
    # 'noxy_noxy-discovery-zookeeper',
    'spring-boot_dot',
    'retrofit_retrofit',
    'retrofit_retrofit-adapters.rxjava',
    'wro4j_wro4j-extensions',
    # 'carbon-apimgt_analyzer-modules.org.wso2.carbon.apimgt.throttling.siddhi.extension',
    'zxing_dot'
]
modu_list = [
    'activiti/activiti_dot',
    'alibaba/fastjson_dot',
    'apache/commons-exec_dot',
    'apache/httpcore_dot',
    'apache/incubator-dubbo_dubbo-remoting.dubbo-remoting-netty',
    'apache/incubator-dubbo_dubbo-rpc.dubbo-rpc-dubbo',
    'davidmoten/rxjava2-extras_dot',
    # 'dromara/hutool_hutool-cron',
    'elasticjob/elastic-job-lite_dot',
    'elasticjob/elastic-job-lite_elastic-job-lite-core',
    # 'espertechinc/esper_examples.rfidassetzone',
    # 'feroult/yawp_yawp-testing.yawp-testing-appengine',
    'flaxsearch/luwak_luwak',
    'fluent/fluent-logger-java_dot',
    'javadelight/delight-nashorn-sandbox_dot',
    'jknack/handlebars.java_dot',
    'joel-costigliola/assertj-core_dot',
    'kagkarlsson/db-scheduler_dot',
    'kevinsawicki/http-request_dot',
    'nationalsecurityagency/timely_server',
    'ninjaframework/ninja_dot',
    'orbit/orbit_dot',
    'qos-ch/logback_dot',
    # 'spinn3r/noxy_noxy-discovery-zookeeper',
    'spring-projects/spring-boot_dot',
    'square/retrofit_retrofit',
    'square/retrofit_retrofit-adapters.rxjava',
    'wro4j/wro4j_wro4j-extensions',
    # 'wso2/carbon-apimgt_analyzer-modules.org.wso2.carbon.apimgt.throttling.siddhi.extension',
    'zxing/zxing_dot']

confs = [
    '19CPU010Mem1GB.sh',
    '20CPU010Mem2GB.sh',
    '21CPU025Mem2GB.sh',
    '22CPU05Mem2GB.sh',
    '23CPU05Mem4GB.sh',
    '24CPU1Mem4GB.sh',
    '25CPU1Mem8GB.sh',
    '26CPU2Mem4GB.sh',
    '27CPU2Mem8GB.sh',
    '28CPU2Mem16GB.sh',
    '29CPU4Mem8GB.sh',
    '01noThrottling.sh']
confs_len = 12

In [ ]:
# CV
for proj in proj_list:
    df = pd.DataFrame(None, columns=['classname', 'methodname', 'conf', 'r1', 'r2', 'r3', 'r4', 'r5',
                                         'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15',
                                         'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25',
                                         'r26', 'r27', 'r28', 'r29', 'r30', 'avg', 'CV'])
    test_loc_dict = {}
    test_tup_dict = {}
    test_avgs_dict = {}
    run300_dict = preproc('preproc_300', proj, 'no')
    cnt = 0
    for k, v in run300_dict.items():
        for itm in v:
            df.loc[len(df.index)] = [k[0], k[1], itm[0], np.nan, np.nan, np.nan, np.nan,
                                     np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
                                     np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
                                     np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
                                     np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
            tup = (k[0], k[1], itm[0])
            test_loc_dict[tup] = cnt
            test_tup_dict[cnt] = tup
            test_avgs_dict[tup] = []
            cnt += 1

    for i in range(30):
        dict_path = f'preproc_10/{i + 1}'
        if proj not in os.listdir(dict_path):
            continue
        run10_dict = preproc(dict_path, proj, 'no')
        for k, v in run10_dict.items():
            idx_arr = []
            avg_arr = []
            for itm in v:
                tup = (k[0], k[1], itm[0])
                idx = test_loc_dict[tup]
                idx_arr.append(idx)
                avg_arr.append(itm[2])
            if len(idx_arr) < confs_len:
                continue
            for j, idx in enumerate(idx_arr):
                df.at[idx, f'r{i + 1}'] = [avg_arr[j]]
                test_avgs_dict[test_tup_dict[idx]].append(avg_arr[j])

    for k, v in test_avgs_dict.items():
        avg = np.mean(v)
        df.at[test_loc_dict[k], 'avg'] = avg
        df.at[test_loc_dict[k], 'CV'] = np.std(v) / avg if avg != 0 else 0
    df.to_csv(f'CV_csvs/{proj}.csv', sep=',', header=True, index=False)

In [4]:
# different machine?
conf_idx_dict = {conf: i for i, conf in enumerate(confs)}
idx_conf_dict = {i: conf for i, conf in enumerate(confs)}
record_df = pd.DataFrame(None, columns=['project', 'different number', 'total test number', 'consistency ratio'])
conf_diff_dict = {}
for proj in proj_list:
    cv_df = pd.read_csv(f'CV_csvs/{proj}.csv')
    df = pd.DataFrame(None, columns=['classname', 'methodname', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6',
                                     'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 
                                     'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25',
                                     'r26', 'r27', 'r28', 'r29', 'r30', 'complete_rounds'], dtype=str)
    
    conf_diff_dict[proj] = 0
    rnd_val_dict = {f'r{i + 1}': [0 for _ in range(confs_len)] for i in range(30)}
    rnd_val_dict['avg'] = [0 for _ in range(confs_len)]
    test_set = set()
    test_cnt = 0
    conf_num = confs_len
    for row in cv_df.itertuples():
        test = (row.classname, row.methodname)
        if test not in test_set:
            df.loc[test_cnt] = [test[0], test[1], None, None, None,
                                     None, None, None, None, None, None,
                                     None, None, None, None, None, None,
                                     None, None, None, None, None, None,
                                     None, None, None, None, None, None,
                                     None, None, None, None]
            test_set.add(test)
            test_cnt += 1
        for i in range(30):
            rnd_cnt = f'r{i + 1}'
            rnd_val_dict[rnd_cnt][conf_idx_dict[row.conf]] = getattr(row, rnd_cnt)
        rnd_val_dict['avg'][conf_idx_dict[row.conf]] = row.avg
        conf_num -= 1
        if conf_num == 0:
            conf_num = confs_len
            test_idx = test_cnt - 1
            is_diff = False
            avg_arr = rnd_val_dict['avg']
            avg_best_conf = idx_conf_dict[avg_arr.index(min(avg_arr))]
            df.at[test_idx, 'complete_rounds'] = avg_best_conf
            for i in range(30):
                rnd_cnt = f'r{i + 1}'
                rnd_arr = rnd_val_dict[rnd_cnt]
                if np.any(np.isnan(rnd_arr)):
                    df.at[test_idx, rnd_cnt] = np.nan
                    continue
                best_conf = idx_conf_dict[rnd_arr.index(min(rnd_arr))]
                df.at[test_idx, rnd_cnt] = best_conf
                if avg_best_conf != best_conf:
                    is_diff = True
            if is_diff:
                conf_diff_dict[proj] += 1
    consistency_ratio = 1 - conf_diff_dict[proj]/test_cnt
    record_df.loc[len(record_df.index)] = [proj, conf_diff_dict[proj], test_cnt, consistency_ratio]
    # df.to_csv(f'best_machine_csvs/{proj}.csv', sep=',', header=True, index=False)
record_df.to_csv('test_number_record.csv', sep=',', header=True, index=False)

In [ ]:
# Runtime = 0?
cnt = [[], []]
for proj in proj_list:
    run10_dict = preproc('preproc_10', proj)
    run300_dict = preproc('preproc_300', proj)
    cnt10 = 0
    cnt300 = 0
    for k, v in run10_dict.items():
        for itm in v:
            if itm[2] == 0:
                cnt10 += 1
                break
    for k, v in run300_dict.items():
        for itm in v:
            if itm[2] == 0:
                cnt300 += 1
                break
    cnt[0].append(cnt10)
    cnt[1].append(cnt300)
print(cnt)

In [ ]:
# Project Information
info_df = pd.DataFrame(None,
                       columns=['id',
                                'project-module',
                                'test_num',
                                'min_failure_rate',
                                'max_failure_rate',
                                'avg_test_runtime'])
for i, proj in enumerate(proj_list):
    preproc_proj_dict = preproc('preproc_300',
                                proj)
    test_num = len(preproc_proj_dict.keys())
    min_fr = 100
    max_fr = 0
    sum_runtime = 0
    num = 0
    for _, val in preproc_proj_dict.items():
        val = np.array(val)
        low_fr = min(val[:, 3].astype('float'))
        high_fr = max(val[:, 3].astype('float'))
        if high_fr > 0:
            for conf in val:
                fr = float(conf[3])
                if fr < min_fr:
                    min_fr = fr 
        sum_runtime += sum(val[:, 2].astype('float'))
        num += len(val)
        if high_fr > max_fr:
            max_fr = high_fr
    info_df.loc[len(info_df.index)] = [
        f'M{i + 1}',
        modu_list[i],
        test_num,
        min_fr if min_fr != 100 else 0.00,
        max_fr,
        sum_runtime / num
    ]
info_df.to_csv('proj_info.csv', sep=',', header=True, index=False)

In [ ]:
# Bruteforce Comparation
import warnings
warnings.filterwarnings("ignore")
ga_path = 'ga_a0.5_ig'
bf_path = 'bf_a0.5_ig'
comp_df = pd.DataFrame(None,
                           columns=['project_id',
                                    'period_ga',
                                    'period_bf',
                                    'trade-off',
                                    'fitness_ratio'])
def get_argu(ser):
    return ser['time_parallel'], ser['price'], ser['fitness'], ser['period']

for i in range(len(proj_list)):
    proj = proj_list[i]
    ga = pd.read_csv(f'rerun_data/{ga_path}/{proj}.csv').iloc[:4, :]
    bf = pd.read_csv(f'rerun_data/{bf_path}/{proj}.csv')
    tot_num = len(bf)
    tot_ga_period = 0
    tot_bf_period = 0
    avg_tradeoff = 0
    avg_fitness_ratio = 0
    for j in range(tot_num):
        ga_itm = ga.iloc[j, :]
        bf_itm = bf.iloc[j, :]
        ga_runtime, ga_price, ga_fitness, ga_period = get_argu(ga_itm)
        bf_runtime, bf_price, bf_fitness, bf_period = get_argu(bf_itm)
        runtime_ratio = ga_runtime / bf_runtime
        price_ratio = ga_price / bf_price
        tot_ga_period += ga_period
        tot_bf_period += bf_period
        avg_tradeoff += runtime_ratio*price_ratio
        avg_fitness_ratio += ga_fitness/bf_fitness
    avg_tradeoff /= tot_num
    avg_fitness_ratio /= tot_num
    comp_df.loc[len(comp_df.index)] = [
        f'M{i + 1}',
        tot_ga_period,
        tot_bf_period,
        avg_tradeoff,
        avg_fitness_ratio]
comp_df.to_csv(f'ga_vs_bf.csv', sep=',', header=True, index=False)
# fitness ratio?

In [ ]:
# Greedy
import json, time
from ast import literal_eval
from itertools import product
from preproc import preproc, conf_prc_map

# Test number -> 14
proj_idx = [4]
comp_df = pd.DataFrame(None,
                       columns=['project_id',
                                'opt-cost_period_ga',
                                'opt-cost_period_bf',
                                'opt-cost_runtime_rate',
                                'opt-cost_price_rate'])
comp_df['opt-time_period_ga'] = pd.Series(dtype='float64')
comp_df['opt-time_period_bf'] = pd.Series(dtype='float64')
comp_df['opt-time_runtime_rate'] = pd.Series(dtype='float64')
comp_df['opt-time_price_rate'] = pd.Series(dtype='float64')

def iter_alloc(a,
               subdir):
    def cal_price(mach_tm_dict):
        price = 0
        for key, per_runtime in mach_tm_dict.items():
            per_price = per_runtime * conf_prc_map[mach_arr[key]] / 3600
            price += per_price
        return price
    
    for cnt, idx in enumerate(proj_idx):
        proj = proj_list[idx]
        resu_path = f'rerun_data/{subdir}/{proj}.csv'
        df = pd.read_csv(resu_path)
        df = df.loc[df['machines_num'] == 4]
        avg_tm_dict = preproc('preproc_300',
                              proj)
        setup_tm_dict = {k: float(0) for k in confs}
        
        confs_dict = literal_eval(df.iloc[0]['confs'])
        n = len(avg_tm_dict.keys())
        arr = np.array([0, 1, 2, 3])
        prods = product(arr, repeat=n)
        mach_arr = []
        for ky, val in confs_dict.items():
            for _ in range(val):
                mach_arr.append(ky)
        tmp_dict = {}
        for ky, val in avg_tm_dict.items():
            tmp_dict[ky] = {}
            for itm in val:
                for i in range(4):
                    if itm[0] == mach_arr[i]:
                        tmp_dict[ky][i] = itm
        idx_ts_map = {i: tst for i, tst in enumerate(list(tmp_dict.keys()))}
        mini = float('inf')
        mini_mach_tm_dict = {}
        t1 = time.time()
        for prod in prods:
            mach_ts_dict = {i: [] for i in range(4)}
            mach_tm_dict = {i: setup_tm_dict[mach_arr[i]] for i in range(4)}
            for idx, mach_id in enumerate(prod):
                tst = idx_ts_map[idx]
                itm = tmp_dict[tst][mach_id]
                mach_ts_dict[mach_id].append(tst)
                mach_tm_dict[mach_id] += itm[2]
            if a == 0:
                prc = cal_price(mach_tm_dict)
                if mini >= prc:
                    mini = prc
                    mini_mach_tm_dict = mach_tm_dict
            else:
                runtm = max(mach_tm_dict.values())
                if mini >= runtm:
                    mini = runtm
                    mini_mach_tm_dict = mach_tm_dict
                
        t2 = time.time()

        ga_period = '<1ms' if df.iloc[0]['period'] < 0.001 else df.iloc[0]['period']
        bf_period = t2 - t1
        runtime_ratio = df.iloc[0]['time_parallel'] / max(mini_mach_tm_dict.values())
        price_ratio = df.iloc[0]['price'] /cal_price(mini_mach_tm_dict)
        if a == 0:
            comp_df.loc[len(comp_df.index)] = [
                proj,
                ga_period,
                bf_period,
                runtime_ratio,
                price_ratio,
                0, 0, 0, 0]
        else:
            comp_df.loc[cnt, 'opt-time_period_ga'] = ga_period
            comp_df.loc[cnt, 'opt-time_period_bf'] = bf_period
            comp_df.loc[cnt, 'opt-time_avg_runtime_rate'] = runtime_ratio
            comp_df.loc[cnt, 'opt-time_avg_price_rate'] = price_ratio
            
        print(ga_period, bf_period, runtime_ratio, price_ratio)

iter_alloc(0, 'ga_a0_ig')
iter_alloc(1, 'ga_a1_ig')
comp_df.to_csv(f'greedy_vs_bf.csv', sep=',', header=True, index=False, float_format='%.2f')

In [ ]:
# {conf1:占比; conf2:占比... ...}
import pandas as pd
time_conf_per_dict = {}
price_conf_per_dict = {}
percentage_df = pd.DataFrame(None, columns=['project', 'runtime_or_price', 'highest_proportion_conf',
                                           'highest_proportion', 'top3_proportion'])

for proj in proj_list:
    time_conf_per_dict[proj] = {conf: 0.0 for conf in confs}
    price_conf_per_dict[proj] = {conf: 0.0 for conf in confs}
    avg_tm_dict = preproc('preproc_300', proj)
    for k, v in avg_tm_dict.items():
        v.sort(key=lambda x:x[2])
        time_conf_per_dict[proj][v[0][0]] += 1
        v.sort(key=lambda x:x[4])
        price_conf_per_dict[proj][v[0][0]] += 1
    time_conf_per_dict[proj] = {conf: time_conf_per_dict[proj][conf]/len(avg_tm_dict.keys()) for conf in confs}
    price_conf_per_dict[proj] = {conf: price_conf_per_dict[proj][conf]/len(avg_tm_dict.keys()) for conf in confs}
    tmp_dict = sorted(time_conf_per_dict[proj].items(), key=lambda x: x[1], reverse=True)
    percentage_df.loc[len(percentage_df.index)] = [proj, 'runtime', 
                                                   tmp_dict[0][0], tmp_dict[0][1], 
                                                   sum(item[1] for item in tmp_dict[:3])]
    tmp_dict = sorted(price_conf_per_dict[proj].items(), key=lambda x: x[1], reverse=True)
    percentage_df.loc[len(percentage_df.index)] = [proj, 'price', 
                                                   tmp_dict[0][0], tmp_dict[0][1], 
                                                   sum(item[1] for item in tmp_dict[:3])]
percentage_df.to_csv(f'percentage_df.csv', sep=',', header=True, index=False)
with open('conf_percentage_record', 'w') as f:
    json.dump(time_conf_per_dict, f, indent=4)
    json.dump(price_conf_per_dict, f, indent=4)

In [ ]:
for proj in proj_list:
    for cnt in [10, 300]:
        df = pd.DataFrame(None, columns=['test', 'conf', 'round_num',
                                         'avg_runtime', 'failrate', 'avg_price'])
        avg_tm_dict = preproc(f'preproc_{cnt}', proj)
        avg_tm_dict = dict(sorted(avg_tm_dict.items(), key=lambda x:x[0]))
        for k, v in avg_tm_dict.items():
            v.sort(key=lambda x:x[0])
            for itm in v:
                df.loc[len(df.index)] = [k, itm[0], itm[1], itm[2], itm[3], itm[4]]
        df.to_csv(f'preproc_csvs/{cnt}/{proj}.csv', sep=',', header=True, index=False)
            